In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#infile = "../NTuples/test_truth.root"
infile = "../NTuples/small_tpceta_sample.root"

# Make Truth DataFrames

## Show Slice DF

In [ ]:

inFile = uproot.open(infile)

#h_tot_pot = inFile["TotalPOT"]
#pot = h_tot_pot.
#print("Total POT", pot)

slc_tree = inFile["slc_tree"]
particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]
daughter_tree1 = inFile["daughter_tree1"]
daughter_tree2 = inFile["daughter_tree2"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")
daughter_df1 = daughter_tree1.arrays(daughter_tree1.keys(), library="pd")
daughter_df2 = daughter_tree2.arrays(daughter_tree2.keys(), library="pd")

slc_df[:2]

## Show Particle1 DF

In [ ]:
particle_df1[:2]

# Show Particle2 DF

In [ ]:
particle_df2[:2]

# Show Daughter1 DF

In [ ]:
daughter_df1[:2]

# Show Daughter2 DF

In [ ]:
daughter_df2[:2]

In [ ]:
def get_slice_npi0(row):
    run = row["run"]
    subrun = row["subrun"]
    evt = row["evt"]
    sel = particle_df1.query("run == "+str(run)+ " and subrun == "+str(subrun) + " and evt == "+str(evt))
    c = sum((sel["pdg"].values == 111))
    return c

slc_df["npi0"] = slc_df.apply(get_slice_npi0, axis=1)
slc_df[:2]

In [ ]:
pdg_colors = {
    # eta
    221:"pink",
    # pi0
    111:"purple",
    # proton
    2212:"red",
    # neutron
    2112:"black",
    # gamma
    22:"gold", 
    # muon
    13:"blue",
    # mu +
    -13:"navy",
    #pi+    
    211:"orangered",
    #pi-
    -211:"magenta",
    # electron
    11:"green",
    # positron
    -11:"cyan"
          
}

pdg_labels = {
    221: r"$\eta$", 
    111: r"$\pi^{0}$",
    2212:"p",
    2112:"n",
    22:r"$\gamma$",
    13:r"$\mu^{-}$",
    211:r"$\pi^{+}$",
    -211:r"$\pi^{-}$",
    11:r"$e^{-}$",
    -11:r"$e^{+}$",
}



In [ ]:
def plot_yz(slice_sel, show_tpc=False, show_crt=False, zoom=False):
    run, subrun, evt, slc = slice_sel[0], slice_sel[1], slice_sel[2], slice_sel[3]
    q = "run == "+str(run)+" and subrun == "+str(subrun) + " and evt == "+str(evt) + "and slc == "+str(slc)
    slc_df_sel = slc_df.query(q)
    part1_sel = particle_df1.query(q)
    part2_sel = particle_df2.query(q)
    daughter1_sel = daughter_df1.query(q)
    daughter2_sel = daughter_df2.query(q)
    plt.title("True Particle Interaction")
    plt.xlabel('True Z [cm]', fontsize=14)
    plt.ylabel('True Y [cm]', fontsize=14)
    #miny, maxy = 10000, -10000
    #minz, maxz = 10000, -10000
    plt.scatter(slc_df_sel["vtx_z"], slc_df_sel["vtx_y"], c="r", label="Primary Vertex")
    pdgs = []
    for i in range(part1_sel.shape[0]):
        r1 = part1_sel.iloc[i]
        r2 = part2_sel.iloc[i]
        pdg = r1["pdg"]
        if pdg not in pdg_colors.keys():
            continue
        y = [r2["start_y"], r2["end_y"]]
        z = [r2["start_z"], r2["end_z"]]
        if pdg not in pdgs:
            plt.plot(z, y, color=pdg_colors[pdg], linewidth=2, label=pdg_labels[pdg])
            pdgs.append(pdg)
        else:
            plt.plot(z, y, color=pdg_colors[pdg])
                
    for i in range(daughter1_sel.shape[0]):
        r1 = daughter1_sel.iloc[i]
        r2 = daughter2_sel.iloc[i]
        pdg = r1["pdg"]
        if pdg not in pdg_colors.keys():
            continue
        y = [r2["start_y"], r2["end_y"]]
        z = [r2["start_z"], r2["end_z"]]
        if pdg not in pdgs:
            plt.plot(z, y, color=pdg_colors[pdg], linewidth=2, label=pdg_labels[pdg])
            pdgs.append(pdg)
        else:
            plt.plot(z, y, color=pdg_colors[pdg])

    if zoom != False:
        plt.xlim(zoom[0])
        plt.ylim(zoom[1])
        
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
    plt.show()
    
print("Made YZ plotting function")        

In [ ]:
# test the YZ plotter

num = 2

slice_sel = [slc_df.iloc[num]["run"],
             slc_df.iloc[num]["subrun"],
             slc_df.iloc[num]["evt"],
             slc_df.iloc[num]["slc"]
            ]


plot_yz(slice_sel)

plot_yz(slice_sel, False, False, [[450, 500], [0, 50]])
plot_yz(slice_sel, False, False, [[275, 325], [0, 50]])

In [ ]:
nopi0_sel = slc_df.query("npi0 == 0")

num_plots = 5

for num in range(num_plots):
    slice_sel = [nopi0_sel.iloc[num]["run"],
                 nopi0_sel.iloc[num]["subrun"],
                 nopi0_sel.iloc[num]["evt"],
                 nopi0_sel.iloc[num]["slc"]
                ]  
    plot_yz(slice_sel)



In [ ]:
slice_sel = [nopi0_sel.iloc[0]["run"],
             nopi0_sel.iloc[0]["subrun"],
             nopi0_sel.iloc[0]["evt"],
             nopi0_sel.iloc[0]["slc"]
            ]  
plot_yz(slice_sel, False, False, [[30, 50], [100, 200]])

In [ ]:

def plot_tpc(ax):

    # Define the corners of the 2D rectangle
    top_corners = [
        [0,   -200, 200],     # Bottom-left corner
        [500, -200, 200],     # Bottom-right corner
        [500,  200, 200],     # Top-right corner
        [0,    200, 200]      # Top-left corner
    ]

    # Create a 3D polygon collection for the rectangle
    top = Poly3DCollection([top_corners], color='red', alpha=0.1, edgecolor='black')

    # Add the rectangle to the 3D plot
    ax.add_collection3d(top)



In [ ]:

def plot_true_slice(run, subrun, evt, slc, e, a, zoom_vertex, tpc, zoom=None):
    q = "run == "+str(run)+" and subrun == "+str(subrun) + " and evt == "+str(evt) + "and slc == "+str(slc) 
    sel1 = particle_df1.query(q)
    sel2 = particle_df2.query(q)
    if sel1.shape[0] != sel2.shape[0]:
        print("PROBLEM: Dataframes have different shapes !!!!!!!!!!!!!")
        return
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('Z Axis [cm]')
    ax.set_ylabel('X Axis [cm]')
    ax.set_zlabel('Y Axis [cm]')
    minx, maxx = 10000, -10000
    miny, maxy = 10000, -10000
    minz, maxz = 10000, -10000
    for i in range(sel1.shape[0]):
        r1 = sel1.iloc[i]
        r2 = sel2.iloc[i]
        pdg = r1["pdg"]
        if pdg != 111:
            continue
        x = [r2["start_x"], r2["end_x"]]
        y = [r2["start_y"], r2["end_y"]]
        z = [r2["start_z"], r2["end_z"]]
        print("x", x, "y", y, "z", z)
        
        if min(x) < minx:
            minx = min(x)
        if max(x) > maxx:
            maxx = max(x)
        if min(y) < miny:
            miny = min(y)
        if max(y) > maxy:
            maxy = max(y)
        if min(z) < minz:
            minz = min(z)
        if max(z) > maxz:
            maxz = max(z)
        
        # eta
        if pdg == 221:
            print("Has Eta !!!")
            ax.plot(z, x, y, color="pink")
        # pi0
        elif pdg == 111:
            print("We have a pi0")
            ax.plot(z, x, y, color="purple")
        # proton
        elif pdg == 2212:
            ax.plot(z, x, y, color="red")
        # neutron
        elif pdg == 2112:
            ax.plot(z, x, y, color="black")
        # gamma
        elif pdg == 22:
            ax.plot(z, x, y, color="yellow")
        # muon
        elif pdg == 13:
            ax.plot(z, x, y, color="blue")
        # piplus or piminus
        elif pdg == 211 or pdg == -211:
            ax.plot(z, x, y, color="orange")
        else:
            print("Unknown PDG", pdg)
            ax.plot(z, x, y, color="green")
    
    ax.view_init(elev=e, azim=a)  # Adjust elevation and azimuth here

    if zoom != None:
        print("Zooming")
        ax.set_xlim(zoom[0])
        ax.set_ylim(zoom[1])
        ax.set_zlim(zoom[2])
    else:
        ax.set_xlim([minz, maxz])
        ax.set_ylim(minx, maxx)
        ax.set_zlim([miny, maxy])

    if zoom_vertex != 0:
        slc_sel_row = slc_df.query(q).iloc[0]
        x = slc_sel_row["vtx_x"]
        y = slc_sel_row["vtx_y"]
        z = slc_sel_row["vtx_z"]
        ax.set_xlim([z - zoom_vertex, z + zoom_vertex])
        ax.set_ylim([x - zoom_vertex, x + zoom_vertex])
        ax.set_zlim([y - zoom_vertex, y + zoom_vertex])

    if tpc:
        plot_tpc(ax)   
        ax.set_xlim([-100, 500])
        ax.set_ylim([-300, 300])
        ax.set_zlim([-300, 300])

    
    plt.show()


plot_true_slice(1.0, 0.0, 290.0, 5.0, 20, 280, 0, 0, [[0, 100], [50, 200], [-100, 100]])



In [ ]:
for num in range(10):
    r = slc_df.iloc[num]
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 20, 280, 0, 1)

In [ ]:
for num in range(10):
    r = slc_df.iloc[num]
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 0, 270, 0, 0)

In [ ]:
for num in range(10):
    r = slc_df.iloc[num]
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 0, 270, 1, 0)

In [ ]:
for num in range(10):
    r = slc_df.iloc[num]
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 90, 270, 0, 1)

In [ ]:
for num in range(20):
    r = slc_df.iloc[num]
    print("run", r["run"], "subrun", r["subrun"], "evt", r["evt"], "slc", r["slc"])
    print("MODE", r["mode"])
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 90, 270, 0, 1)

In [ ]:
three_pi0 = slc_df.query("npi0 == 3")

print(three_pi0.shape[0])

In [ ]:
for num in range(three_pi0.shape[0]):
    r = three_pi0.iloc[num]
    plot_true_slice(r["run"], r["subrun"], r["evt"], r["slc"], 20, 300, 0, 0)